# Final Project
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__  
Throughout this course you’ve engaged with key principles required to develop scalable machine learning analyses for structured and unstructured data. Working in Hadoop Streaming and Spark you’ve learned to translate common machine learning algorithms into Map-Reduce style implementations. You’ve developed the ability to evaluate Machine Learning approaches both in terms of their predictive performance as well as their scalability. For the final project you will demonstrate these skills by solving a machine learning challenge on a new dataset. Your job is to perform Click Through Rate prediction on a large dataset of Criteo advertising data made public as part of a Kaggle competition a few years back. As you perform your analysis, keep in mind that we are not grading you on the final performance of your model or how ‘advanced’ the techniques you use but rather on your ability to explain and develop a scalable machine learning approach to answering a real question.

More about the dataset:
https://www.kaggle.com/c/criteo-display-ad-challenge

# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [59]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [61]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [62]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "fproj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [25]:
# create a data directory (RUN THIS CELL AS IS)
!mkdir data

mkdir: cannot create directory `data': File exists


In [ ]:
# grab the tar.gz file from kaggle
#!curl https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz -o data/dac.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 24 4364M   24 1060M    0     0   714k      0  1:44:14  0:25:19  1:18:55     0

In [5]:
# I couldn't get this to unpack the tarball, so I just did it in Windows
#!tar -xvz data/dac.tar.gz

^C


In [72]:
# take a look at the to the training data set
!head -n 2000 data/train > data/train200

In [73]:
!head -100 data/train | tail -n 5 

10015211672544614902,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,42606fe6,cb0fb677,1,0,17037,320,50,1934,2,39,-1,16
10015376300289320595,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,03108db9,a0f5f879,1,0,15701,320,50,1722,0,35,100084,79
10015405794859644629,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,0b697be1,1f0bc64f,1,0,15701,320,50,1722,0,35,100084,79
10015629448289660116,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,58db4f0c,6332421a,1,0,15708,320,50,1722,0,35,-1,79
100156980486870304,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,02b9b0fc,1aa0e912,1,0,15706,320,50,1722,0,35,-1,79


In [27]:
# load the data into Spark RDD for convenience of use later (RUN THIS CELL AS IS)
projectRDD = sc.textFile('data/train')
projectRDD.count()

40428968

In [28]:
#get all categorical variables
def get_categorical(df):
    categorical = [var for var in df.columns if df[var].dtype=='O']
    return categorical
#get all numerical variables
def get_numerical(df):
    numerical = [var for var in df.columns if df[var].dtype!='O']
    
    return numerical

numeric_features = get_numerical(train_df)
print(numeric_features)

NameError: name 'train_df' is not defined

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
parse_date = lambda val :pd.datetime.strptime(val,'%y%m%d%H')
train_df = pd.read_csv('data/train2000',parse_dates=['hour'],date_parser=parse_date)
corrmat = train_df.corr()
k = 25
cols = corrmat.nlargest(k,'click')['click'].index
print(cols)
cm = np.corrcoef(train_df[cols].values.T)
sns.set(font_scale=1.25)
plt.figure(figsize = (20,10))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, 
                 yticklabels=cols.values, xticklabels=cols.values)
plt.show()



In [ ]:
import random
import numpy as n
import pandas as pd
import gzip



In [ ]:
train['click'].value_counts()/len(train)
train.head()


In [ ]:
train.hour.describe()

In [ ]:
train.C1.value_counts()/len(train)

In [ ]:
import os
import sys
import os.path
import pyspark
#import urllib2

#response = urllib2.urlopen('https://www.dropbox.com/s/rf64jk6eufmserm/dac_sample.txt?dl=1')

#dacContents = response.read().split('\n')
file = open('data/train2000', 'r') 
dacContents = file.read().split("\n") 
dacContents = [x.strip().replace('\t', ',') for x in dacContents]

numPartitions = 2
rawData = sc.parallelize(dacContents)

In [ ]:
weights = [.8, .1, .1]
seed = 42
rawTrainData, rawValidationData, rawTestData = rawData.randomSplit(weights,seed)
# Cache the data
rawTrainData.cache()
rawValidationData.cache()
rawTestData.cache()

nTrain = rawTrainData.count()
nVal = rawValidationData.count()
nTest = rawTestData.count()
print( nTrain, nVal, nTest, nTrain + nVal + nTest)
print (rawData.take(3))

In [ ]:
def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    
    list_features_all = point.split(",")
    list_final = []
    n = len(list_features_all)
    for i in range(1,n):
      list_final.append((i-1,list_features_all[i]))
    
    return list_final

parsedTrainFeat = rawTrainData.map(parsePoint)

numCategories = (parsedTrainFeat
                 .flatMap(lambda x: x)
                 .distinct()
                 .map(lambda x: (x[0], 1))
                 .reduceByKey(lambda x, y: x + y)
                 .sortByKey()
                 .collect())
print(numCategories)

print (numCategories[2][1])

In [ ]:
from collections import defaultdict
import hashlib

def hashFunction(numBuckets, rawFeats, printMapping=False):
    """Calculate a feature dictionary for an observation's features based on hashing.

    Note:
        Use printMapping=True for debug purposes and to better understand how the hashing works.

    Args:
        numBuckets (int): Number of buckets to use as features.
        rawFeats (list of (int, str)): A list of features for an observation.  Represented as
            (featureID, value) tuples.
        printMapping (bool, optional): If true, the mappings of featureString to index will be
            printed.

    Returns:
        dict of int to float:  The keys will be integers which represent the buckets that the
            features have been hashed to.  The value for a given key will contain the count of the
            (featureID, value) tuples that have hashed to that key.
    """
    mapping = {}
    for ind, category in rawFeats:
        featureString = category + str(ind)
        #signature = hashlib.sha256(h.encode('utf-8')).hexdigest()
        #mapping[featureString] = int(int(hashlib.md5(featureString).hexdigest(), 16) % numBuckets)
        mapping[featureString] = int(int(hashlib.md5(featureString.encode('utf-8')).hexdigest(), 16) % numBuckets)
        
    if(printMapping): print (mapping)
    sparseFeatures = defaultdict(float)
    for bucket in mapping.values():
        sparseFeatures[bucket] += 1.0
    return dict(sparseFeatures)

In [ ]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
def parseHashPoint(point, numBuckets):
    """Create a LabeledPoint for this observation using hashing.

    Args:
        point (str): A comma separated string where the first value is the label and the rest are
            features.
        numBuckets: The number of buckets to hash to.

    Returns:
        LabeledPoint: A LabeledPoint with a label (0.0 or 1.0) and a SparseVector of hashed
            features.
    """
    
    list_features_all = point.split(",")
    list_final = []
    n = len(list_features_all)
    for i in range(1,n):
      list_final.append((i-1,list_features_all[i]))
   
   
    sp = SparseVector(numBuckets,hashFunction(numBuckets, list_final, True))
    
    if (list_features_all[0] == "0"):
      lbp = LabeledPoint(0,sp)
    else:
      lbp = LabeledPoint(1,sp)
  
    return lbp

numBucketsCTR = 2 ** 15
hashTrainData = rawTrainData.map(lambda x: parseHashPoint(x, numBucketsCTR))
hashTrainData.cache()
hashValidationData = rawValidationData.map(lambda x: parseHashPoint(x, numBucketsCTR))
hashValidationData.cache()
hashTestData = rawTestData.map(lambda x: parseHashPoint(x, numBucketsCTR))
hashTestData.cache()

#signature = hashlib.sha256(h.encode('utf-8')).hexdigest()
print( hashTrainData.take(4))

In [ ]:
def evaluateResults(model, data):
    """Calculates the log loss for the data given the model.

    Args:
        model (LogisticRegressionModel): A trained logistic regression model.
        data (RDD of LabeledPoint): Labels and features for each observation.

    Returns:
        float: Log loss for the data.
    """
    
    return data.map(lambda x: computeLogLoss(getP(x.features, model.weights, model.intercept),x.label)).mean()

In [ ]:
from math import log

def computeLogLoss(p, y):
    """Calculates the value of log loss for a given probabilty and label.

    Note:
        log(0) is undefined, so when p is 0 we need to add a small value (epsilon) to it
        and when p is 1 we need to subtract a small value (epsilon) from it.

    Args:
        p (float): A probabilty between 0 and 1.
        y (int): A label.  Takes on the values 0 and 1.

    Returns:
        float: The log loss value.
    """
    epsilon = 10e-12
    
    if p==0:
      p+=epsilon
    elif p==1:
      p-=epsilon
    
    if y==1:
      logloss = -log(p)
    elif y==0:
      logloss = -log(1-p)
    
    return logloss

In [ ]:
from math import exp #  exp(-t) = e^-t

def getP(x, w, intercept):
    """Calculate the probability for an observation given a set of weights and intercept.

    Note:
        We'll bound our raw prediction between 20 and -20 for numerical purposes.

    Args:
        x (SparseVector): A vector with values of 1.0 for features that exist in this
            observation and 0.0 otherwise.
        w (DenseVector): A vector of weights (betas) for the model.
        intercept (float): The model's intercept.

    Returns:
        float: A probability between 0 and 1.
    """
    
    rawPrediction = x.dot(w) + intercept
    
    # Bound the raw prediction value
    rawPrediction = min(rawPrediction, 20)
    rawPrediction = max(rawPrediction, -20)
    return 1/(1+exp(-rawPrediction))

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
numIters = 500
regType = 'l2'
includeIntercept = True

# Initialize variables using values from initial model training
bestModel = None
bestLogLoss = 1e10

In [ ]:
stepSizes = [1,10]
regParams = [1e-6,1e-3]
for stepSize in stepSizes:
    print("#stepSize",stepSize)
    for regParam in regParams:
        model = (LogisticRegressionWithSGD
                 .train(hashTrainData, numIters, stepSize, regParam=regParam, regType=regType,
                        intercept=includeIntercept))
        logLossVa = evaluateResults(model, hashValidationData)
        print ('\tstepSize = {0:.1f}, regParam = {1:.0e}: logloss = {2:.3f}'
               .format(stepSize, regParam, logLossVa))
        if (logLossVa < bestLogLoss):
            bestModel = model
            bestLogLoss = logLossVa

print(type(bestLogLoss))
print(bestLogLoss)
#print ('Hashed Features Validation Logloss:\n\tBaseline = {0:.3f}\n\tLogReg = {1:.3f}'
print ('Hashed Features Validation Logloss:\tLogReg = {}'
       .format(bestLogLoss))
#       .format(logLossValBase, bestLogLoss))
#stepSize = 1.0, regParam = 1e-06: logloss = 0.470
#stepSize = 1.0, regParam = 1e-03: logloss = 0.470
#stepSize = 10.0, regParam = 1e-06: logloss = 0.446
#stepSize = 10.0, regParam = 1e-03: logloss = 0.448
#Hashed Features Validation Logloss:
#Baseline = 0.523
#LogReg = 0.446

In [86]:
from pyspark.ml.feature import FeatureHasher
from pyspark.sql import SQLContext, Row
from pyspark.mllib.classification import LogisticRegressionWithSGD

regParams = [1e-6,1e-3]
sqlContext = SQLContext(sc)
df = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").load("data/train2000")
df.printSchema()

hasher = FeatureHasher(inputCols=["id", "click", "hour", "C1","banner_pos","site_id","site_domain"],
                       outputCol="features")

testDF = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").load("data/train_tail_10")
testDF = testDF.select("id", "hour", "C1","banner_pos","site_id","site_domain")
hashertest = FeatureHasher(inputCols=["id", "hour", "C1","banner_pos","site_id","site_domain"],
                       outputCol="features")
ftestDF = hashertest.transform(testDF)


featurized = hasher.transform(df)
df.show(2)
featurized.show(2)
featurized.select('features').show(5,truncate=False)


numIters = 5
regType = 'l1'
includeIntercept = True
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.types import IntegerType
def parse(line):
    print(line)
    click = line.click
    print(type(click))
    c = eval(click)
    print(c,type(c))
    
    h = line.features
    print(type(h),h)
  
    #yield LabeledPoint(c,h)
    #sp = SparseVector(7,h)
    #yield LabeledPoint(c,h)
    print("Sp",h)
    yield LabeledPoint(c,h)
    
#dataRDD = df.flatMap(patse)
#dataRDD.take(10)
stepSizes = [1,10]

f = featurized.select('click','features')
f.show(10)
f = f.withColumn("click", f["click"].cast(IntegerType()))
#xx = f.rdd.flatMap(parse)
#xx.take(3)

#yy = f.rdd.map(lambda row: LabeledPoint(row.click, row.features))
#yy.collect()

#from pyspark.mllib.utils import MLUtils
#df = MLUtils.convertVectorColumnsFromML(f, "features")
rdd = xx
#model = LogisticRegressionWithSGD.train(rdd, numIters, stepSize, regParam=regParam, regType=regType,
#                       intercept=includeIntercept)
#print(model)

from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="click",featuresCol="features",maxIter = 10)
lrModel = lr.fit(f)
print(lrModel)
predictions = lrModel.transform(ftestDF)
predictions.show(10)

testDF.show(10)


root
 |-- id: string (nullable = true)
 |-- click: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- C1: string (nullable = true)
 |-- banner_pos: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- device_conn_type: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)

+--------------------+-----+--------+----+----------+--------+-

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

# Question 1: Question Formulation 
Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

# Question 2: Algorithm Explanation
Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithym that you will perform.

# Question 3: EDA & Discussion of Challenges
Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform

In [11]:
average_CTR = projectRDD.map(lambda x: int(x.split('\t')[0])).mean()

In [12]:
average_CTR

0.25622338372976045

# Question 4: Algorithm Implementation 
Develop a 'homegrown' implementation of the algorithn, apply it to the training dataset and evaluate your results on the test set. 

# Question 5: Application of Course Concepts
Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts. 